In [35]:
import requests
import os
from flask import (
    Flask,
    jsonify,
    request,
    render_template,
    Response,
    send_from_directory,
)
from markupsafe import Markup

from utils import ph

app = Flask(__name__)


@app.route("/")
def index():
    return render_template("index.html")


@app.route("/favicon.ico")
def favicon():
    return send_from_directory(
        os.path.join(app.root_path, "static"),
        "favicon.ico",
        mimetype="image/vnd.microsoft.icon",
    )

def show_files(path, filter, all_files):
    file_list = os.listdir(path)
    for file in file_list:
        cur_path = os.path.join(path, file)
        if os.path.isdir(cur_path):
            show_files(cur_path, filter, all_files)
        else:
            if file.endswith(filter):
                all_files.append(path.replace(
                    "\\", "/").replace("./", "") + "/" + file)
    return all_files

def getInput(func, meta=None):
    result = []
    co, dft = func.__code__, func.__defaults__
    dft = (
        dict(zip(co.co_varnames[co.co_argcount -
                                len(dft): co.co_argcount], dft))
        if dft
        else {}
    )
    for varname in co.co_varnames[: co.co_argcount]:
        v = ph.dynamic()
        v.name = varname
        v.value = dft[v.name] if v.name in dft else None
        v.dv = ph.jh.Encode(dft[v.name]) if v.value is not None else ""
        v.type = type(v.value).__name__ if v.value is not None else "str"
        result.append(v)
    if meta and meta.input:
        tmp = dict([(i.name, i) for i in meta.input])
        for p in result:
            if p.name in tmp:
                p = ph.dd.extend(p, tmp[p.name])
    return result


def getOutput(func, meta=None):
    if not meta:
        return []
    return meta.output or []

@app.route("/<name>.md")
def readme(name):
    if name == "README":
        return Response(ph.dd.file2str("api/main.md"), mimetype="text/x-markdown")
    if name == "_sidebar":
        files = show_files("./", ".md", [])
        files = [file.split("/") for file in files]
        sidebar = list(set([".".join(file[1:4])
                            for file in files if len(file) == 5]))
        sidebar.sort()
        content = "\n".join(
            [
                "- [{0}]({1}.md)".format(item.upper(), item)
                for item in sidebar if item != "main.md"
            ]
        )
        return Response(content, mimetype="text/x-markdown")
    name = "/".join(name.split("."))
    path = "./api/" + name
    files = show_files(path, ".md", [])
    docs = []
    for file in files:
        model = ph.dynamic()
        arr = file.split("/")
        if len(arr) < 5:
            continue
        arr = arr[1:4]
        model.name = arr[-1]
        model.key = ".".join(arr)
        model.path = "api/{0}/main.py".format("/".join(arr))
        model.md = ph.dd.file2str(file)
        model.json = ph.dd.file2str(file.replace(".md", ".json"))
        func = ph.require(model.path)
        model.odef = ph.jh.Decode(model.json or "{}")
        model.input = getInput(func, model.odef)
        model.output = getOutput(func, model.odef)
        model.tests = model.odef.tests or []
        for t in model.tests:
            t.data = t.data or ph.dynamic()
            if t.url:
                t.a = t.data.a
                t.i = Markup(
                    t.data.i if isinstance(
                        t.data.i, str) else ph.jh.Encode(t.data.i)
                )
                t.json = ph.jh.Encode(t.data or {})
            else:
                t.a = model.key
                t.url = "/model/rpc/call"
                t.json = ph.jh.Encode(t.data or {})
                t.i = Markup(t.json)
        model.hasInput = bool(model.input)
        model.hasOutput = bool(model.output)
        model.hasTest = bool(model.tests)
        model.showkey = True
        docs.append(render_template("api.md", model=model))
    content = "\n\n".join(docs)
    return Response(content, mimetype="text/x-markdown")


@app.route("/model/rpc/call", methods=["POST"])
def model_rpc_call():
    a = request.form["a"]
    i = request.form["i"]
    return Response(ph.jh.Encode(call(a, i)), mimetype="application/json")

def call(a, i):
    """
    根据指定的路径和输入参数调用目标函数，并返回结果。

    参数:
    a (str): 要调用的函数的路径，使用点号（.）分隔。例如 "module.submodule.function"。
    i (str or dict): 传递给函数的输入参数，可以是 JSON 字符串或已解码的字典对象。

    返回:
    目标函数的返回结果。
    """
    # 将点号分隔的路径转换为斜杠分隔的路径
    key = "/".join(a.split("."))

    # 构建目标函数所在的 Python 文件路径
    path = "api/{0}/main.py".format(key)

    # 动态加载指定路径的 Python 文件，并获取其中导出的函数
    func = ph.require(path)

    # 如果输入参数是字符串，则将其解码为 Python 对象；否则直接使用
    input = ph.jh.Decode(i) if isinstance(i, str) else i

    # 调用加载的函数，并传递解码后的输入参数
    return ph.dd.run(func, input)


app.run(host="0.0.0.0", port=8081)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8081
 * Running on http://10.0.0.64:8081
Press CTRL+C to quit
127.0.0.1 - - [19/Mar/2025 17:49:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:49:55] "GET /README.md HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:49:55] "GET /_sidebar.md HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:49:55] "GET /model.demo.add.md HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:49:56] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:49:57] "GET /favicon.ico HTTP/1.1" 304 -
127.0.0.1 - - [19/Mar/2025 17:50:01] "POST /model/rpc/call HTTP/1.1" 200 -


3


127.0.0.1 - - [19/Mar/2025 17:53:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:53:06] "GET /model.demo.add.md HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:53:06] "GET /_sidebar.md HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2025 17:53:07] "GET /favicon.ico HTTP/1.1" 304 -
127.0.0.1 - - [19/Mar/2025 17:53:07] "GET /favicon.ico HTTP/1.1" 304 -
